In [33]:
import os
import pandas as pd
import numpy as np
import openai
from openai.embeddings_utils import distances_from_embeddings

In [34]:
# Create an empty dict 
content_list = []

# Specify the folder path where the .txt files are located
folder_path = '.'

# List all files in the folder
files = os.listdir(folder_path)

# Filter .txt files
txt_files = [file for file in files if file.endswith('.txt')]

# Loop through each .txt file and perform operations
for txt_file in txt_files:
    # Create the complete file path by joining folder path and file name
    file_path = os.path.join(folder_path, txt_file)
    
    # Open and read the file
    with open(file_path, 'r') as file:
        file_content = file.read()
        content_list.append({"name": txt_file.replace(".txt", ""), "content": file_content})

In [35]:
# create a dataframe
df = pd.DataFrame(content_list)
df["embeddings"] = ""
df

,name,content,embeddings
0,Empowering Your Health Journey - Essential Tip...,Consider signing up for the Northwind Health P...,
1,Exploring Your Health Insurance Options - A Gu...,Welcome to Contoso Electronics! We are excited...,
2,Maximizing Your Health Coverage - Navigating I...,HOW PROVIDERS AFFECT YOUR COSTS\nIn-Network Pr...,
3,Understanding Exclusions - Your Guide to North...,EXCLUSIONS\nAlthough Northwind Health Plus pro...,
4,Unlocking the Benefits - Navigating Prescripti...,Prescription Drug\nCOVERED SERVICES: Prescript...,


In [36]:
# prepare open ai calls
openai.api_type = "azure"
openai.api_key = "3483d96f18af44c5b4da5328865a8fe9"
openai.api_base = "https://aoaidalabbe.openai.azure.com/"
openai.api_version = "2022-12-01"

embbeding_model = "text-embedding-ada-002"
temperature = 0

In [37]:
# Get the Embeddings
for index, row in df.iterrows():
    df.at[index, "embeddings"] = openai.Embedding.create(input         = row["content"], 
                                                         model         = embbeding_model, 
                                                         deployment_id = embbeding_model, 
                                                         temperature   = temperature)["data"][0]["embedding"]

In [38]:
# The resulting dataframe will be like this
df.head()

,name,content,embeddings
0,Empowering Your Health Journey - Essential Tip...,Consider signing up for the Northwind Health P...,"[-0.012706773355603218, -0.007550304289907217,..."
1,Exploring Your Health Insurance Options - A Gu...,Welcome to Contoso Electronics! We are excited...,"[0.0010583240073174238, -0.0029922700487077236..."
2,Maximizing Your Health Coverage - Navigating I...,HOW PROVIDERS AFFECT YOUR COSTS\nIn-Network Pr...,"[0.005240269936621189, 0.0003642811207100749, ..."
3,Understanding Exclusions - Your Guide to North...,EXCLUSIONS\nAlthough Northwind Health Plus pro...,"[-0.0031386762857437134, -0.001379720168188214..."
4,Unlocking the Benefits - Navigating Prescripti...,Prescription Drug\nCOVERED SERVICES: Prescript...,"[0.012051036581397057, -0.009269006550312042, ..."


In [39]:
def query_similar_content(database, query, n_results = 3):

    embedded_query = openai.Embedding.create(input         = query, 
                                             model         = embbeding_model, 
                                             deployment_id = embbeding_model, 
                                             temperature   = temperature)["data"][0]["embedding"]

    df_distances = database.copy(deep = True)
    df_distances["distance_from_query"] = distances_from_embeddings(embedded_query, database["embeddings"])
    return df_distances[["name", "distance_from_query"]].sort_values("distance_from_query")[:n_results]

In [40]:
query_similar_content(df, "sign up for health insurance")

,name,distance_from_query
0,Empowering Your Health Journey - Essential Tip...,0.181789
1,Exploring Your Health Insurance Options - A Gu...,0.203258
3,Understanding Exclusions - Your Guide to North...,0.214729


In [41]:
query_similar_content(df, "refill prescription medicine")

,name,distance_from_query
4,Unlocking the Benefits - Navigating Prescripti...,0.200822
0,Empowering Your Health Journey - Essential Tip...,0.247938
3,Understanding Exclusions - Your Guide to North...,0.263451


In [42]:
query_similar_content(df, "services not covered by the insurance")

,name,distance_from_query
3,Understanding Exclusions - Your Guide to North...,0.170785
4,Unlocking the Benefits - Navigating Prescripti...,0.220713
1,Exploring Your Health Insurance Options - A Gu...,0.234809
